In [2]:
# Importar librerías para tratamiento de datos
import pandas as pd  

# Importar librerías para web scraping y manipulación de datos
from bs4 import BeautifulSoup
import requests
import re
import random

# Importar librerías para automatización de navegadores web con Selenium
from selenium import webdriver  
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager  
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import Select  

# Importar librerías para pausar la ejecución
from time import sleep  

# Configuraciones
pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.

import warnings
warnings.filterwarnings('ignore')

In [3]:
df_actor = pd.read_csv("union_actores_unicos.csv")
dict_actores_limpio= {"name": df_actor["name"].to_list(), "link":[]}

for e in df_actor["link"]:
    try:
        dict_actores_limpio["link"].append(e.replace("'","").split(",")[1])
    except:
        dict_actores_limpio["link"].append(e)


df_actor_limpio = pd.DataFrame(dict_actores_limpio)
df_actor_limpio

,name,link
0,Joanna Cole,https://www.imdb.com/name/nm2051894/?ref_=tt_...
1,Stanley Wojculewski,https://www.imdb.com/name/nm1798696/?ref_=tt_...
2,Melanie Stablein,https://www.imdb.com/name/nm1762923/?ref_=tt_...
3,Max Baloian,https://www.imdb.com/name/nm0051062/?ref_=tt_...
4,Keith Davies,https://www.imdb.com/name/nm1795121/?ref_=tt_...
...,...,...
167296,Welismar Silva,https://www.imdb.com/name/nm14430550/?ref_=tt...
167297,welton silva,https://www.imdb.com/name/nm14430554/?ref_=tt...
167298,Makoto Kurami,https://www.imdb.com/name/nm5729926/?ref_=tt_...
167299,Hiroshi Miki,https://www.imdb.com/name/nm3855245/?ref_=tt_...


In [8]:

##Busqueda info actores 
options = webdriver.FirefoxOptions()
options.page_load_strategy = "eager"
driver = webdriver.Firefox(options)
datos_actores= {"Nombre":[], "Nacimiento":[], "Conocido":[], "Que hace":[], "Premios":[], "Oscar": []}

indice_inicio = 0
contador = indice_inicio
indice_final = 5
intentos = 0 
antiguo_contador = 0

try:
    while contador <= indice_final:

        try:

            for i in range(len(df_actor_limpio))[contador:indice_final+1]:
                
                driver.get(df_actor_limpio.iloc[i]["link"])
                driver.maximize_window()
                
                sleep(1)
                    # Aceptar las cookies PAG IMDB
                try:
                    driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()
                    print("cookies accepted")

                except:
                    print("cookies haven't been accepted")

                datos_actores["Nombre"].append(df_actor_limpio.iloc[i]["name"])

                    # Buscar la edad
                try:
                    busqueda_anio = driver.find_elements("css selector",'div[data-testid="birth-and-death-birthdate"]:last-child > span:last-child')[-1].text
                    anio = busqueda_anio.split()
                    datos_actores["Nacimiento"].append(anio[-1])
                    
                except:
                    datos_actores["Nacimiento"].append("Desconocido")

                #Buscamos porque es conocido

                try:
                    conocido_por = driver.find_elements("css selector",'div[data-testid="nm_flmg_kwn_for"] > div > div:nth-child(2) > div > div > div > a')

                    lista_conocido = []
                    
                    for x in conocido_por:
                        lista_conocido.append(x.text)

                    datos_actores["Conocido"].append(lista_conocido)
                    
                except:
                    datos_actores["Conocido"].append("Desconocido")

                #Buscamos que hace
                try:
                    profesiones = driver.find_elements("css selector",'ul.ipc-inline-list.ipc-inline-list--show-dividers.sc-d8941411-2.cdJsTz.baseAlt > li')
                    lista_profesiones = []

                    for profesion in profesiones:
                        lista_profesiones.append(profesion.text)
                    
                    datos_actores["Que hace"].append(lista_profesiones)

                    
                except:
                    datos_actores["Que hace"].append("Desconocido")
                    
                #Buscamos los prmios
                
                try:
                    premios = driver.find_element("css selector",'div[data-testid="awards"] > ul > li > div > ul > li > span').text
                    datos_actores["Premios"].append(premios)

                except:
                    datos_actores["Premios"].append("Desconocido")

                #Buscamos los prmios OSCAR
                
                try:
                    oscars = driver.find_element("css selector",'div[data-testid="awards"] > ul > li > a').text
                    if "oscar" not in oscars.lower():
                        datos_actores["Oscar"].append("--")
                    else:
                        datos_actores["Oscar"].append(oscars)

                except:
                    datos_actores["Oscar"].append("--")
            
                contador +=1 


        except:
            print(f"ha fallado la pelicula {contador}")
            if antiguo_contador == contador:
                intentos += 1
                if intentos == 4:
                    sleep(20)
                    intentos = 0
                    driver.back()
            else:
                intentos += 1
                antiguo_contador = contador    
except:
    print("El codigo ha fallado, motivo desconocido")

    
df_actores_info = pd.DataFrame(datos_actores)
df_actores_info.to_csv(f"EXTRACCION REBE/df_actores_info_{indice_inicio}_{indice_final}.csv")
    


cookies accepted
cookies haven't been accepted
cookies haven't been accepted
cookies haven't been accepted
cookies haven't been accepted
cookies haven't been accepted
